# Export MNIST

read images and labels in ubyte format

In [16]:
import gzip
import numpy as np
import struct

images_path = 'mnist/t10k-images-idx3-ubyte.gz'
labels_path = 'mnist/t10k-labels-idx1-ubyte.gz'
out_path = 'mnist/test/'

# images_path = 'mnist/train-images-idx3-ubyte.gz'
# labels_path = 'mnist/train-labels-idx1-ubyte.gz'
# out_path = 'mnist/train/'

# https://stackoverflow.com/questions/39969045/parsing-yann-lecuns-mnist-idx-file-format

with gzip.open(images_path,'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    images = np.frombuffer(f.read(), dtype=np.dtype(np.uint8).newbyteorder('>'))
    images = images.reshape((size, nrows, ncols))


with gzip.open(labels_path,'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    labels = np.frombuffer(f.read(), dtype=np.dtype(np.uint8).newbyteorder('>'))

define helper function to generate polygon for the images

In [13]:
import cv2

def image2poly(image, img_size = 28, max_size = 10):
    cnt, _ = cv2.findContours(np.array(image), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = max(cnt, key=cv2.contourArea)
    poly = cv2.approxPolyDP(cnt, 0.015 * cv2.arcLength(cnt, True), True) / img_size
    poly = poly.reshape(-1, 2)[:max_size]

    poly = np.c_[np.arange(1, len(poly) + 1) / (len(poly) + 1), poly]
    y = np.array([[0, 0, 0]])
    z = np.array([[1, 0, 0]])
    poly = np.r_[y, poly, z]
    for i in range(0, 12 - len(poly)):
        poly = np.r_[poly, z]

    if (len(poly) != 12):
        print("Diff")
    return poly

export images to png

In [17]:
from PIL import Image
from os.path import join

dataset_entries = []

idx = 0
for image, label in zip(images, labels):
    im = Image.fromarray(image)
    file_path = join("images", f"img{idx:05}_{label}.png")
    poly = image2poly(image).flatten().tolist()
    poly = ','.join(str(n) for n in poly)

    dataset_entries.append({
        "file_path": file_path,
        "label": label,
        "polygon": poly
    })

    im.save(join(out_path, file_path))
    idx += 1

save label and polygons to csv with an association to the file path they refer to

In [18]:
import csv

with open(join(out_path, "polygon-mnist.csv"), 'w', newline='') as file: 
    writer = csv.DictWriter(file, fieldnames = ["file_path", "label", "polygon"])
    writer.writeheader() 
    writer.writerows(dataset_entries)